In [1]:
import os
import pandas as pd
import numpy as np
import random
import pickle
# import torch

import warnings
warnings.filterwarnings('ignore')

%config Completer.use_jedi = False
working_dir = '/home/thuytt/test_bert/NCKH/'
os.chdir(working_dir)

In [12]:
os.environ["CUDA_VISIBLE_DEVICES"]="7"
print('cuda')

cuda


In [36]:
df_6kuser = pd.read_csv('./data/6kuser/data_6kuser_sapo.csv')
df_6kuser

,uid,articleID,sapo
0,1006923096,2191240,Đầu xe mình chạm cạnh dưới biển số xe trước là...
1,1014147481,2191240,Đầu xe mình chạm cạnh dưới biển số xe trước là...
2,1007506304,2191240,Đầu xe mình chạm cạnh dưới biển số xe trước là...
3,1002615050,2207856,Ngoài nguyên nhân uống rượu-bia hay cố tình bỏ...
4,1014147481,2207856,Ngoài nguyên nhân uống rượu-bia hay cố tình bỏ...
...,...,...,...
713058,1004228557,4248348,HLV Alexandre Polking bố trí Lee Nguyễn trong ...
713059,1043162603,4248367,Romelu Lukaku và Lautaro Martinez cùng ghi bàn...
713060,1002624858,4248376,"Lần đầu tiên, giống lợn ỉ có nguy cơ tuyệt chủ..."
713061,1062314309,4248376,"Lần đầu tiên, giống lợn ỉ có nguy cơ tuyệt chủ..."


In [5]:
# load data_dict 
file = open('./model/data_dict/user6k_articles_dict.pkl', 'rb')
data_dict = pickle.load(file)
file.close()

In [6]:
def _get_all_user_articles(data_dict):
    all_user_articles = []
    for uid in data_dict:
        articles = data_dict[uid]
        for x in articles:
            all_user_articles.append(x)
    all_user_articles = list(set(all_user_articles))
    print('number unique articles', len(all_user_articles))
    return all_user_articles
all_user_articles = _get_all_user_articles(data_dict)

number unique articles 54692


In [7]:
# lấy user mà clicked articleID => article khác của user khác
import random

def get_negative_sample(articleID, all_user_articles, npratio=4):
    negative_articles = all_user_articles.copy()
    negative_articles.remove(articleID)
#     print(len(negative_articles))
    negative_4_articles = random.sample(negative_articles, npratio)
    return [articleID] + list(negative_4_articles)

In [8]:
def preprocess_clicked(data_dict, npratio=4):
    uids = list(data_dict.keys())
    user_count = len(uids)
    userid_dict = {}
    for uid in uids:
        if uid not in userid_dict:
            userid_dict[uid] = len(userid_dict) # map uid_raw -> 0 1 2

    all_user_articles = _get_all_user_articles(data_dict)

    all_train_id = []
    all_train_pn = []
    all_label = []

    all_test_id = []
    all_test_pn = []
    all_test_label = []
    all_test_index = []

    all_user_pos = []
    all_test_user_pos = []

    for uid in uids:
        clicked_news = data_dict[uid]  # get all unique article which user clicked

        for idx in range(len(clicked_news)-1):
            articleID = clicked_news[idx]
            all_train_pn.append(get_negative_sample(articleID, all_user_articles))
            all_label.append([1,0,0,0,0])
            all_train_id.append(userid_dict[uid])

            remain_clicked = clicked_news.copy()
            remain_clicked.remove(articleID)
            remain_clicked = random.sample(remain_clicked, min(50, len(remain_clicked)))
            remain_clicked += [0] * (50-len(remain_clicked)) # <50 cho = 0
            all_user_pos.append(remain_clicked)

        # get the last line for testing
        sess_index = []
        sess_index.append(len(all_test_pn))

        article_test = all_user_articles[-1]
        all_test_pn += get_negative_sample(article_test, all_user_articles)
        sess_index.append(len(all_test_pn))
        all_test_index.append(sess_index)
        all_test_label += [1,0,0,0,0]
        all_test_id += [userid_dict[uid]] * (npratio+1)
        allpos = random.sample( clicked_news, min(50, len(clicked_news)))
        allpos += [0] * (50-len(allpos))
        for i in range(5):
            all_test_user_pos.append(allpos)
    
    all_train_pn = np.array(all_train_pn,dtype='int32')
    all_label = np.array(all_label,dtype='int32')
    all_train_id = np.array(all_train_id,dtype='uint64')
    all_test_pn = np.array(all_test_pn,dtype='int32')
    all_test_label = np.array(all_test_label,dtype='int32')
    all_test_id = np.array(all_test_id,dtype='uint64')
    all_user_pos = np.array(all_user_pos,dtype='int32')
    all_test_user_pos = np.array(all_test_user_pos, dtype='int32')

    return (userid_dict, user_count, all_train_pn, all_label, all_train_id, all_test_pn, all_test_label, all_test_id, all_user_pos, all_test_user_pos, all_test_index)

In [9]:
%%time
userid_dict,user_count, all_train_pn, all_label, all_train_id, all_test_pn, all_test_label, all_test_id, all_user_pos, all_test_user_pos, all_test_index = preprocess_clicked(data_dict)

number unique articles 54692
CPU times: user 3min 35s, sys: 300 ms, total: 3min 36s
Wall time: 3min 36s


In [20]:
dataloader = (userid_dict,user_count, all_train_pn, all_label, all_train_id, all_test_pn, all_test_label, all_test_id, all_user_pos, all_test_user_pos, all_test_index)

file = open('./model/thuytt_ver2.2/dataloader.pkl', 'wb')
pickle.dump(dataloader, file)
file.close()

## Tokenizer and make word_dict

In [10]:
from vncorenlp import VnCoreNLP
VnCoreNLP_jar_file = '../vncorenlp/VnCoreNLP-1.1.1.jar'
rdrsegmenter = VnCoreNLP(VnCoreNLP_jar_file, annotators='wseg')
embedding_dim=768

In [11]:
df_articles_with_sapo = df_6kuser[df_6kuser['articleID'].isin(all_user_articles)].drop_duplicates(subset='articleID')
df_articles_with_sapo

,uid,articleID,sapo
0,1006923096,2191240,Đầu xe mình chạm cạnh dưới biển số xe trước là...
8,1002611667,3101986,Tôi đã mơ đến những cái xa hơn với em nhưng bi...
10,1002611667,3102220,Em có nên nói cho bạn trai hiện tại biết em từ...
11,1002633452,3102941,"Trong điều kiện bình thường, quả bóng và lông ..."
16,1002611667,3103609,Khi nóng giận anh sẵn sàng mạt sát tôi dù tôi ...
...,...,...,...
713033,1045108384,4248332,"Ngày 14/3, du khách nườm nượp đổ về khiến Ban ..."
713049,1011365931,4248341,Tiền đạo HAGL Nguyễn Công Phượng ngã trong vòn...
713054,1039680178,4248345,HLV Kiatikusk muốn các cầu thủ HAGL cải thiện ...
713057,1049676180,4248348,HLV Alexandre Polking bố trí Lee Nguyễn trong ...


In [12]:
articles_with_sapo = dict()
for index, row in df_articles_with_sapo.iterrows():
    articleID = row['articleID']
    sapo = row['sapo']
    articles_with_sapo[articleID] = sapo
    
articles_with_sapo[articleID]

'Lần đầu tiên, giống lợn ỉ có nguy cơ tuyệt chủng được các nhà khoa học của Viện Chăn nuôi nhân bản thành công.'

In [28]:
file = open('./data/6kuser/articles_with_sapo.pkl', 'wb')
pickle.dump(articles_with_sapo, file)
file.close()

In [13]:
def preprocess_news(articles_sapo_dict):
    articleIds = list(articles_sapo_dict.keys())
    sapos = list(articles_sapo_dict.values())

    news = {} 

    for i in range(len(articleIds)):
        if articleIds[i] not in news:
            tokenized_words = rdrsegmenter.tokenize(sapos[i])[0]
            news[articleIds[i]] = tokenized_words

    
    word_dict_raw = {'PADDING': [0,999999]}
    for articleId in news:
        for word in news[articleId]:
            if word.isspace(): # handle word ==' '
                continue
            if word in word_dict_raw:
                word_dict_raw[word][1] += 1 # increase freq
            else:
                word_dict_raw[word] = [len(word_dict_raw), 1] # format: [index, freq]
                
    word_dict = {}
    for i in word_dict_raw:
        if word_dict_raw[i][1] >= 2:
            word_dict[i] = [len(word_dict), word_dict_raw[i][1]]
    print('len word_dict (freq>=2 vs raw):', len(word_dict), len(word_dict_raw)) # chỉ để so sánh (loại bỏ freq =1)
    
    print('leng news (tokenizer):',len(news))

    news_words = [ [0]*30 ] # 
    news_index = {0:0}
    
    for articleId in news: # quét các article
        word_id = []
        news_index[articleId] = len(news_index)
        for word in news[articleId]: # quét các tokens
            if word in word_dict:
                word_id.append(word_dict[word][0])
        word_id = word_id[:30] # lấy word_id của article (embedd)
        news_words.append(word_id + [0]*(30-len(word_id))) # max 30 tokens, <30 cho =0
    
    news_words = np.array(news_words, dtype='int32')

    return word_dict, news_words, news_index, news

In [35]:
%%time
word_dict, news_words, news_index, news = preprocess_news(articles_with_sapo)

len word_dict (freq>=2 vs raw): 24281 57229
leng news (tokenizer): 54692
CPU times: user 2min, sys: 19 s, total: 2min 19s
Wall time: 3min 20s


In [43]:
file = open('./model/thuytt_ver2.2/phobert_news_preprocess.pkl', 'wb')
pickle.dump((word_dict, news_words, news_index, news), file)
file.close()

## Phobert + embeddingMatrix

In [14]:
import argparse

from fairseq.data import Dictionary

In [15]:
vocab = Dictionary()
vocab.add_from_file('../PhoBERT_base_transformers/dict.txt')

In [16]:
# load bert model
from fairseq.models.roberta import RobertaModel
import os
phobert = RobertaModel.from_pretrained('../PhoBERT_base_fairseq', checkpoint_file='model.pt')
phobert.eval()  # disable dropout (or leave in train mode to finetune)

# Incorporate the BPE encoder into PhoBERT-base 
from fairseq.data.encoders.fastbpe import fastBPE  
from fairseq import options  
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes', 
    default="../PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()
phobert.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

In [40]:
def get_embedding(word_dict):
    embedding_dict = {}

    for word in word_dict:
        input_ids = vocab.encode_line(word, append_eos=False, add_if_not_exist=False).long()
        embedding_tensor = phobert.extract_features(input_ids)
        embedding_dict[word] = embedding_tensor.data.cpu().numpy()[0][0]
        
    embedding_matrix = [0]*len(word_dict)
    cand = []

    for i in embedding_dict:
        embedding_matrix[word_dict[i][0]] = np.array(embedding_dict[i], dtype='float32')
        cand.append(embedding_matrix[word_dict[i][0]])
    
    cand = np.array(cand, dtype='float32')
    mu = np.mean(cand, axis=0)
    Sigma = np.cov(cand.T)
    norm = np.random.multivariate_normal(mu, Sigma, 1)

    for i in range(len(embedding_matrix)):
        if type(embedding_matrix[i]) == int: # unknown words
            embedding_matrix[i] = np.reshape(norm, embedding_dim)
    
    embedding_matrix[0] = np.zeros(embedding_dim, dtype='float32')
    embedding_matrix = np.array(embedding_matrix, dtype='float32')

    print(embedding_matrix.shape)
    return embedding_matrix

In [41]:
%%time
embedding_mat = get_embedding(word_dict)

(24281, 768)
CPU times: user 6h 27min 55s, sys: 9min 48s, total: 6h 37min 44s
Wall time: 16min 49s


In [42]:
file = open('./model/thuytt_ver2.2/phobert_embed_mat.pkl', 'wb')
pickle.dump(embedding_mat, file)
file.close()

## load preprocess data

In [3]:
# load thuytt_ver2.2

file = open('./model/thuytt_ver2.2/dataloader.pkl', 'rb')
userid_dict, user_count, all_train_pn, all_label, all_train_id, all_test_pn, all_test_label, all_test_id, all_user_pos, all_test_user_pos, all_test_index = pickle.load(file)
file.close()

file = open('./model/thuytt_ver2.2/phobert_news_preprocess.pkl', 'rb')
word_dict, news_words, news_index, news = pickle.load(file)
file.close()

file = open('./model/thuytt_ver2.2/phobert_embed_mat.pkl', 'rb')
embedding_mat = pickle.load(file)
file.close()
embedding_dim=768

## Model

### Metric

In [4]:
def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)

### function for generate batch

In [5]:
def _articles_to_index(arr_np):
    arr_index_articles = []
    for arr_articles in arr_np:
        index_article = []
        for article in arr_articles:
            index_article.append(news_index[article])
        arr_index_articles.append(index_article)
    arr_index_articles = np.array(arr_index_articles, dtype='int32')
    return arr_index_articles

def news_word2index(word_dict, sapo):
    new_words = []
    news_tokenizer = rdrsegmenter.tokenize(sapo)[0]
    
    word_id = []
    for word in news_tokenizer: # quét các tokens
        if word in word_dict:
            word_id.append(word_dict[word][0])
    word_id = word_id[:30] # lấy word_id của article (embedd)
    news_words = (word_id + [0]*(30-len(word_id))) # max 30 tokens, <30 cho =0
    
    return news_words

In [6]:
def generate_batch_data_train(all_train_pn,all_label,all_train_id,all_user_pos,batch_size):
    inputid = np.arange(len(all_label))
    np.random.shuffle(inputid)
    y=all_label
    batches = [inputid[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            if(i.size ==0):
                continue
            index_all_train_pn = _articles_to_index(all_train_pn[i])
            candidate = news_words[index_all_train_pn]
            candidate_split=[candidate[:,k,:] for k in range(candidate.shape[1])]

            #
            index_all_user_pos = _articles_to_index(all_user_pos[i])
            browsed_news=news_words[index_all_user_pos]
            browsed_news_split=[browsed_news[:,k,:] for k in range(browsed_news.shape[1])]
            userid=np.expand_dims(all_train_id[i],axis=1)
            label=all_label[i]
            yield (candidate_split +browsed_news_split+[userid], label)

In [7]:
def generate_batch_data_test(all_test_pn,all_test_label,all_test_id,all_test_user_pos,batch_size):
    inputid = np.arange(len(all_test_label))
    y=all_test_label
    batches = [inputid[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            if(i.size ==0):
                continue
            index_all_test_pn = [news_index[x] for x in all_test_pn[i]]
            candidate = news_words[index_all_test_pn]

            browsed_news=news_words[_articles_to_index(all_test_user_pos[i])]
            browsed_news_split=[browsed_news[:,k,:] for k in range(browsed_news.shape[1])]
            userid=np.expand_dims(all_test_id[i],axis=1)
            label=all_test_label[i]
    
            yield ([candidate]+ browsed_news_split+[userid], label)

### Model 

In [8]:
import keras
from keras.layers import *
from keras.models import Model
from keras import backend as K
from keras.optimizers import *
from sklearn.metrics import roc_auc_score
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

Using TensorFlow backend.


In [9]:
MAX_SENT_LENGTH=30
MAX_SENTS=50 # maximum clicked news for user embedding
npratio=4
batch_size=32
n_epoch=2
user_count

5981

In [26]:
class NPA:
    def __init__(self, num_users, max_sents, max_sent_length, 
                 embedding_mat, npratio = 4, dense_dim = 200, lr = 0.00025):
        self.__num_users = num_users
        self.__max_sents = max_sents
        self.__max_sent_length = max_sent_length
        self.__embedding_mat = embedding_mat
        
        embedding_dim = embedding_mat.shape[1]
        self.__embedding_dim = embedding_dim
        self.__dense_dim = dense_dim
        self.__npratio = npratio
        
        self.__lr = lr
        
        # User embeddings
        self.user_input = Input(shape=(1,), dtype='uint64')
        print('user_input:',self.user_input)
        user_embedding_layer = Embedding(self.__num_users, self.__max_sents, trainable=True) #ouput leng == max clicked_news
        user_embedding = user_embedding_layer(self.user_input)
        
        user_embedding_word = Dense(self.__dense_dim, activation='relu')(user_embedding)
        user_embedding_word = Flatten()(user_embedding_word)
        
        # dense for user, tinh candiate 
        self.user_embedd_dense = Dense(self.__embedding_dim, activation='tanh')(user_embedding_word)
        
        self.user_embedd = Model(self.user_input, user_embedding_word)
        
        user_embedding_news = Dense(self.__dense_dim, activation='relu')(user_embedding)
        user_embedding_news = Flatten()(user_embedding_news)
        
        
        # News embedding 
        self.news_input = Input(shape=(self.__max_sent_length,), dtype='int32')
        print(self.news_input)
        embedding_layer = Embedding(self.__embedding_mat.shape[0], self.__embedding_dim, 
                                    weights=[self.__embedding_mat],trainable=True) # ?? Co train hay giu nguyen?
        embedded_sequences = embedding_layer(self.news_input)
        embedded_sequences =Dropout(0.2)(embedded_sequences)
        
        cnn_layer = Conv1D(padding='same', activation='relu', strides=1, 
                          filters = self.__embedding_dim, kernel_size=3)(embedded_sequences)
        cnnoutput = Dropout(0.2)(cnn_layer)
        
        # thuytt vector article save DB
        self.new_embedding_extractor = Model(self.news_input, cnnoutput)
        
        attention_a = Dot((2, 1))([cnnoutput, Dense(self.__embedding_dim, activation='tanh')(user_embedding_word)])
        attention_weight = Activation('softmax')(attention_a)
        news_rep = keras.layers.Dot((1, 1))([cnnoutput, attention_weight])
        news_encoder = Model([self.news_input, self.user_input], news_rep)
        
        self.news_encoder = news_encoder
        '''
        define layer to get newsEncoder from CNNoutput + userID
        '''
        




        # clicked news embedding
        clicked_news_input = [keras.Input((self.__max_sent_length,), dtype='int32') for _ in range(MAX_SENTS)]
        clicked_news_rep = [news_encoder([news, self.user_input]) for news in clicked_news_input]
        clicked_news_rep = concatenate([Lambda(lambda x: K.expand_dims(x, axis=1))(news) for news in clicked_news_rep],axis=1)

        # User Embedding
        attention_news = keras.layers.Dot((2, 1))(
            [clicked_news_rep, Dense(self.__embedding_dim, activation='tanh')(user_embedding_news)])
        attention_weight_news = Activation('softmax')(attention_news)
        user_rep = keras.layers.Dot((1, 1))([clicked_news_rep, attention_weight_news])
        

        # thuytt vector user
        self.user_extractor = Model(clicked_news_input + [self.user_input], user_rep)
    
        # candidate news embedding
        candidate_news_input = [keras.Input((self.__max_sent_length,), dtype='int32') for _ in range(1 + self.__npratio)]
        candidate_vecs = [news_encoder([candidate, self.user_input]) for candidate in candidate_news_input]

        
        # Click probability
        logits = [keras.layers.dot([user_rep, candidate_vec], axes=-1) for candidate_vec in candidate_vecs]
        logits = keras.layers.Activation(keras.activations.softmax)(keras.layers.concatenate(logits))

        self.trainer = Model(candidate_news_input + clicked_news_input + [self.user_input], logits)
        self.trainer.compile(loss='categorical_crossentropy', optimizer = Adam(self.__lr), metrics=['acc'])
        
        candidate_one_vec = news_encoder([self.news_input, self.user_input])
        candidate_score = Activation(keras.activations.sigmoid)(keras.layers.dot([user_rep, candidate_one_vec], axes=-1))
        self.evaluator = Model([self.news_input] + clicked_news_input + [self.user_input], candidate_score)
        
    def train(self, data_generator, steps, verbose=True):
        self.trainer.fit_generator(data_generator, epochs=1, steps_per_epoch=steps, verbose=verbose)
        
    def evaluate(self, data_generator, steps, verbose=True):
        score = self.evaluator.predict_generator(data_generator, steps=steps, verbose=verbose)
        return score


IndentationError: expected an indented block (<ipython-input-26-80ef028442bf>, line 64)

In [27]:
npa = NPA(user_count, MAX_SENTS, MAX_SENT_LENGTH, embedding_mat)
npa

user_input: Tensor("input_61:0", shape=(None, 1), dtype=uint64)
Tensor("input_62:0", shape=(None, 30), dtype=int32)


ValueError: This model has not yet been built. Build the model first by calling build() or calling fit() with some data. Or specify input_shape or batch_input_shape in the first layer for automatic build. 

In [16]:
npa.news_encoder.get_weights()

[array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-0.40647736, -0.14589363,  0.3054583 , ...,  0.62434244,
         -0.34173533, -0.41065878],
        [-0.41325927, -0.1851263 ,  0.32863775, ...,  0.6448328 ,
         -0.34047505, -0.39708027],
        ...,
        [-0.41384056, -0.17363083,  0.25468487, ...,  0.6153544 ,
         -0.26381645, -0.3773931 ],
        [-0.41384056, -0.17363083,  0.25468487, ...,  0.6153544 ,
         -0.26381645, -0.3773931 ],
        [-0.4349298 , -0.1733819 ,  0.23069884, ...,  0.60135376,
         -0.23096177, -0.38317263]], dtype=float32),
 array([[ 0.02487693,  0.00885804,  0.03872306, ..., -0.01213701,
         -0.01514735, -0.02216547],
        [ 0.00999272,  0.00260042, -0.01861709, ...,  0.02974359,
          0.01127516,  0.01659801],
        [ 0.02752041, -0.03599731,  0.02843152, ..., -0.04738287,
         -0.03604595,  0.01042125],
        ...,
        [-0.04398825,  0.00168482, -0.0

### Train

In [17]:
# metrhistoryt of [auc, mrr, ndcg@5, ndcg@10]
best_metric = [0., 0., 0., 0.]

In [18]:
len(all_test_id)
len(all_train_id)

206193

In [21]:
len(all_train_id)//batch_size
len(all_test_id)//batch_size

934

In [22]:
# full n_ecoch
results = [] 
import time
n_epoch = 1
start = time.time()
for ep in range(n_epoch):
    start_i = time.time()
    print('Epoch:', ep)
    traingen=generate_batch_data_train(all_train_pn, all_label, all_train_id, all_user_pos, batch_size)
    npa.train(traingen, steps=10)
    testgen=generate_batch_data_test(all_test_pn, all_test_label, all_test_id, all_test_user_pos, batch_size)
    click_score = npa.evaluate(testgen, steps=25, verbose=1)

    all_auc=[]
    all_mrr=[]
    all_ndcg=[]
    all_ndcg2=[]
    for m in all_test_index:
        if np.sum(all_test_label[m[0]:m[1]])!=0 and m[1]<len(click_score):
            all_auc.append(roc_auc_score(all_test_label[m[0]:m[1]], click_score[m[0]:m[1],0]))
            all_mrr.append(mrr_score(all_test_label[m[0]:m[1]], click_score[m[0]:m[1],0]))
            all_ndcg.append(ndcg_score(all_test_label[m[0]:m[1]], click_score[m[0]:m[1],0],k=5))
            all_ndcg2.append(ndcg_score(all_test_label[m[0]:m[1]], click_score[m[0]:m[1],0],k=10))
    results.append([np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2)])
    
    metric = [np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2)]
    if metric[0] > best_metric[0] and np.mean(metric) > np.mean(best_metric):
        best_metric = metric
        print('Best model: True')
        print(metric)
    else:
        print('Best model: False')
        print(f'AUC: {metric[0]:.4f}')
    end_i = time.time()
    print('Time training each epoch:', (end_i-start_i)/60)

end = time.time()
print('Total time training: ', (end-start)/60)

Epoch: 0
Epoch 1/1
25/25 [==============================] - 3s 125ms/step
Best model: True
[0.8176100628930818, 0.7046121593291406, 0.7803790699583834, 0.7803790699583834]
Time training each epoch: 1.2533535559972127
Total time training:  1.2533722281455995


In [23]:
# test
npa.news_encoder.get_weights()

[array([[ 1.8394842e-03, -1.2912512e-03,  2.0356765e-03, ...,
         -1.3314268e-03, -3.5754434e-04,  1.7547706e-03],
        [-4.0570900e-01, -1.4668530e-01,  3.0616632e-01, ...,
          6.2527174e-01, -3.4265968e-01, -4.1148153e-01],
        [-4.1399369e-01, -1.8418668e-01,  3.2956946e-01, ...,
          6.4427412e-01, -3.4059763e-01, -3.9612976e-01],
        ...,
        [-4.1384056e-01, -1.7363083e-01,  2.5468487e-01, ...,
          6.1535442e-01, -2.6381645e-01, -3.7739310e-01],
        [-4.1384056e-01, -1.7363083e-01,  2.5468487e-01, ...,
          6.1535442e-01, -2.6381645e-01, -3.7739310e-01],
        [-4.3492979e-01, -1.7338189e-01,  2.3069884e-01, ...,
          6.0135376e-01, -2.3096177e-01, -3.8317263e-01]], dtype=float32),
 array([[ 0.02487693,  0.00885804,  0.03872306, ..., -0.01213701,
         -0.01514735, -0.02216547],
        [ 0.00999272,  0.00260042, -0.01861709, ...,  0.02974359,
          0.01127516,  0.01659801],
        [ 0.02752041, -0.03599731,  0.02843152

In [74]:
# save NPA subModel
newEmbeddExtractor_model = npa.new_embedding_extractor # save CNNoutput for article
userEmbedd_model = npa.user_embedd # save userid -> Dense 
userExtractor_model = npa.user_extractor # -> save Model predict userVector 
trainer_model = npa.trainer # -> save Model train data

newEmbeddExtractor_model.save('./model/thuytt_ver2.2/new_extractor_model')
userEmbedd_model.save('./model/thuytt_ver2.2/user_embedd_model')
userExtractor_model.save('./model/thuytt_ver2.2/user_extractor')
trainer_model.save('./model/thuytt_ver2.2/npa_train_model')

## Optimizer speed

In [50]:
file = open('./model/thuytt_ver2.1/articles_rep10k_raw.pkl', 'rb')
artiles_test = pickle.load(file)
len(artiles_test)

10000

In [54]:
tmp = list(artiles_test.keys())
articles_0 = artiles_test[tmp[0]]
articles_1 = artiles_test[tmp[1]]

In [83]:
articles_represent = np.array([articles_0, articles_1])
articles_represent

array([[[[0.        , 0.        , 0.        , ..., 0.        ,
          0.44010243, 0.2914183 ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.01015337, ..., 0.01342217,
          0.0008333 , 0.04235733],
         [0.        , 0.        , 0.01015337, ..., 0.01342217,
          0.0008333 , 0.04235733],
         [0.        , 0.        , 0.00506613, ..., 0.00440978,
          0.00539138, 0.01715868]]],


       [[[0.        , 0.        , 0.        , ..., 0.        ,
          0.52750474, 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.2710826 , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0

In [96]:
articles_represent = np.array([articles_0, articles_1]).reshape(2,30,768)
articles_represent.shape

(2, 30, 768)

In [71]:
userid = np.array([1], dtype='uint64')
userid_embedd = npa.user_embedd.predict(userid)

In [93]:
class NewsEncoder:
    def __init__(self, embedding_mat):
        print('Initialize NewsEncoder_Model\n')
        self.__embedding_dim = embedding_mat.shape[1]
        
        userid_dense_input = Input(shape=(200,), dtype='float32')
        news_cnn_input = Input(shape=(30,768,), dtype='float32')
        attention_a = Dot((2, 1))([news_cnn_input, Dense(self.__embedding_dim, activation='tanh')(userid_dense_input)])
        attention_weight = Activation('softmax')(attention_a)
        news_rep = keras.layers.Dot((1, 1))([news_cnn_input, attention_weight])
        self.news_encoder = Model([news_cnn_input, userid_dense_input], news_rep)
newsEncoder = NewsEncoder(embedding_mat)

Initialize NewsEncoder_Model



In [98]:
print('article:', articles_0.shape)
print('user:', userid_embedd.shape)
input_newsEncoder = [articles_0, userid_embedd]
newsEncoder.news_encoder.predict(input_newsEncoder)

article: (1, 30, 768)
user: (1, 200)


array([[0.00000000e+00, 0.00000000e+00, 1.86209613e-03, 8.23845970e-04,
        2.30014091e-03, 7.83509202e-03, 2.71041878e-03, 1.71504635e-03,
        9.06086876e-04, 1.22758420e-03, 1.85917551e-03, 3.37907206e-03,
        2.03895615e-03, 2.31310190e-03, 0.00000000e+00, 3.85517115e-03,
        2.17793416e-03, 4.78125969e-03, 1.73536281e-03, 3.16022383e-03,
        8.65946524e-04, 1.56262447e-03, 1.78362522e-03, 2.64515891e-03,
        4.32355469e-03, 4.86126263e-03, 2.81871832e-03, 5.08878333e-03,
        7.68172788e-03, 4.03586589e-03, 2.78349384e-03, 2.93452423e-02,
        2.05890508e-03, 2.62589217e-03, 2.14984640e-03, 1.41256698e-03,
        3.99998482e-03, 2.41633574e-03, 2.58293329e-03, 0.00000000e+00,
        4.02087532e-03, 0.00000000e+00, 1.00449147e-03, 5.45489602e-03,
        2.04439485e-03, 2.67541292e-03, 0.00000000e+00, 1.90019293e-03,
        4.11074900e-04, 2.97127804e-03, 2.74102297e-03, 1.41945225e-03,
        3.82030150e-03, 5.16796717e-03, 4.56104695e-04, 1.748934

In [121]:
def __candidate_rep(arr_articleID, candidates):
# load input article_rep for score
    multiple_inputs_newsEncoder = [] # need to load in memory
    for articleID in arr_articleID:
        print('load input rep:', articleID)
        # print('article convert:', article_represent.shape)
        article_represent = candidates[articleID]
        input_newsEncoder = [article_represent, userid_embedd]
        candidate_rep = newsEncoder.news_encoder.predict(input_newsEncoder)
        multiple_inputs_newsEncoder.append(candidate_rep)
    print(len(multiple_inputs_newsEncoder))
    
    return multiple_inputs_newsEncoder
def _pool_calcualte_uids(sample_candidate, cores, func):
    import numpy as np
    from multiprocessing import Pool

    n_cores = cores
    pool = Pool(n_cores)
    
    arr_articleID = list(sample_candidate.keys())
    uids_split = np.array_split(arr_articleID, n_cores)
    arguments = []
    for i in range(n_cores):
        arg = (uids_split[i], sample_candidate)
        arguments.append(arg)
    print('something pool4')
    result_list = pool.starmap(func, arguments)
    print('something pool2')
    return result_list
tmp = _pool_calcualte_uids(artiles_test, cores=4, func=__candidate_rep)

OSError: [Errno 12] Cannot allocate memory

In [132]:
number_of_articles = 2
class Multiple_Score:
    def __init__(self, number_articles):
        print('Initialize Multiple_Score_Model\n')
        user_rep = Input(shape=(768,), dtype='float32')
        candidates = [Input(shape=(768,), dtype='float32') for _ in range(number_articles)]
        logits = [keras.layers.dot([user_rep, candidate_vec], axes=-1) for candidate_vec in candidates]
        logits = keras.layers.Activation(keras.activations.sigmoid)(keras.layers.concatenate(logits))
        inputs_tt = candidates + [user_rep]
        self.Score = Model(candidates+[user_rep], logits)

multiScore_model = Multiple_Score(number_of_articles)

class NewsEncoder:
    def __init__(self, embedding_mat, number_articles):
        print('Initialize NewsEncoder_Model\n')
        self.__embedding_dim = embedding_mat.shape[1]
        
        userid_dense_input = Input(shape=(200,), dtype='float32')
        news_cnn_input = [Input(shape=(30,768,), dtype='float32') for _ in range(number_articles)]
        attention_a = Dot((2, 1))([news_cnn_input, Dense(self.__embedding_dim, activation='tanh')(userid_dense_input)])
        attention_weight = Activation('softmax')(attention_a)
        news_rep = keras.layers.Dot((1, 1))([news_cnn_input, attention_weight])
        self.news_encoder = Model(news_cnn_input + [userid_dense_input], news_rep)
newsEncoder = NewsEncoder(embedding_mat, number_of_articles)

Initialize Multiple_Score_Model

Initialize NewsEncoder_Model



ValueError: Layer dot_44 was called with an input that isn't a symbolic tensor. Received type: <class 'list'>. Full input: [[<tf.Tensor 'input_151:0' shape=(None, 30, 768) dtype=float32>, <tf.Tensor 'input_152:0' shape=(None, 30, 768) dtype=float32>], <tf.Tensor 'dense_18/Tanh:0' shape=(None, 768) dtype=float32>]. All inputs to the layer should be tensors.

In [128]:
articles_represent = [articles_0, articles_1]
input_newsEncoder = articles_represent + [userid_embedd]
newsEncoder.news_encoder.predict(input_newsEncoder)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 3 arrays: [array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.44010243, 0.2914183 ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
       ...

## Test new NPA

In [223]:
class NewsEncoder:
    def __init__(self):
        print('Initialize NewsEncoder_Model\n')
        userid_dense_input = Input(shape=(200,), dtype='float32')
        news_cnn_input = Input(shape=(30,768,), dtype='float32')
        output = npa.myLayer([news_cnn_input, userid_dense_input])
        news_encoder = Model([news_cnn_input,userid_dense_input], output)
newsEncoder = NewsEncoder()

Initialize NewsEncoder_Model

<class 'list'>
Tensor("input_797:0", shape=(None, 30, 768), dtype=float32)
Tensor("input_796:0", shape=(None, 200), dtype=float32)


ValueError: Layer dense_180 was called with an input that isn't a symbolic tensor. Received type: <class 'numpy.ndarray'>. Full input: [array([[3.33802286e-03, 2.30505597e-03, 6.16974570e-03, 7.72028789e-03,
        1.53624360e-02, 0.00000000e+00, 0.00000000e+00, 9.63102619e-04,
        0.00000000e+00, 1.12829311e-02, 1.84341962e-03, 0.00000000e+00,
        0.00000000e+00, 4.56311012e-04, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 5.66846831e-03, 3.50920372e-02, 5.26562473e-03,
        0.00000000e+00, 2.60322280e-02, 0.00000000e+00, 0.00000000e+00,
        1.50277736e-02, 1.67842098e-02, 0.00000000e+00, 6.20627031e-03,
        0.00000000e+00, 0.00000000e+00, 1.60705717e-03, 1.15591707e-02,
        3.63207771e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.59879755e-02, 3.96605441e-03, 0.00000000e+00, 3.02311648e-02,
        2.93663098e-03, 2.68063694e-02, 1.39663266e-02, 3.04859076e-02,
        1.06040388e-02, 0.00000000e+00, 1.18773300e-02, 4.01255488e-03,


In [218]:
npa.myLayer(input_newsEncoder)

ValueError: Layer mylayer_4 was called with an input that isn't a symbolic tensor. Received type: <class 'numpy.ndarray'>. Full input: [array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.44010243, 0.2914183 ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.01015337, ..., 0.01342217,
         0.0008333 , 0.04235733],
        [0.        , 0.        , 0.01015337, ..., 0.01342217,
         0.0008333 , 0.04235733],
        [0.        , 0.        , 0.00506613, ..., 0.00440978,
         0.00539138, 0.01715868]]], dtype=float32), array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.52750474, 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.2710826 , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
   